In [1]:
'''For preprocessing images'''
import numpy as np
from PIL import Image
import scipy
import matplotlib.pyplot as plt
import csv
import glob
'''For CNN'''
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
from matplotlib.pyplot import imshow

Using TensorFlow backend.


In [13]:
def standardize(img):
    # padding
    longer_side = max(img.size)
    horizontal_padding = (longer_side - img.size[0]) / 2
    vertical_padding = (longer_side - img.size[1]) / 2
    img = img.crop(
        (
            -horizontal_padding,
            -vertical_padding,
            img.size[0] + horizontal_padding,
            img.size[1] + vertical_padding
        )
    )
    # resizing to standardized size
    img = img.resize([256,256],Image.ANTIALIAS) \
    # plt.imshow(img) # To see the image being standardized.
    
    # converting image to numpy array
    img.load()
    img = np.asarray(img, dtype="int32")
    return img

In [14]:
def function():
    for filename in glob.glob('input/subset_data/train/*.tif'):
        img =Image.open(filename)
        img = standardize(img)
        print(img.shape)
        return

In [15]:
function()

(256, 256, 3)


In [16]:
'''Loading data'''
def get_id_from_filename(filename):
    id = filename.split("\\")[-1]
    id = id.split(".")[0]
    return id

## Edit Code:
- If number of training example changes

In [17]:
def load_train():
    names = []
    # Change first number base on number of training examples
    X_train = np.empty((450,256,256,3), dtype="int32")
    Y_train = np.empty(shape=(450,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/train/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_train[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_train[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_train[names.index(name)] = np.array([0,1]) # means 1
    return X_train,Y_train

## Edit Code:
- If number of training example changes

In [28]:
def load_test():
    names = []
    # Change first number base on number of training examples
    X_test = np.empty((50,256,256,3), dtype="int32")
    Y_test = np.empty(shape=(50,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/test_with_outputs/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_test[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_test[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_test[names.index(name)] = np.array([0,1]) # means 1
    return X_test,Y_test

In [19]:
X_train, Y_train = load_train()
X_test, Y_test = load_test()
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)
# To check values inside.
# print(X_train)
# print(Y_train)
# print(X_test)
# print(Y_test)

X_train shape:  (450, 256, 256, 3)
Y_train shape:  (450, 2)
X_test shape:  (50, 256, 256, 3)
Y_test shape:  (50, 2)


In [20]:
def TrainingModel(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(8, (4, 4), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((8, 8), name='max_pool0')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(16, (2, 2), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool1')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(2, activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='trainingModel')

    return model

In [23]:
trainingModel = TrainingModel((X_train.shape[1],X_train.shape[2],X_train.shape[3]))

In [24]:
trainingModel.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

## Simple CNN
- set epoch to 50 and record the accuracy

In [25]:
trainingModel.fit(x = X_train, y = Y_train, epochs = 50, batch_size = 32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
450/450 [==============================] - 43s 95ms/step - loss: 0.8821 - acc: 0.5489


In [26]:
preds = trainingModel.evaluate(x = X_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

50/50 [==============================] - 2s 41ms/step

Loss = 0.723783688545227
Test Accuracy = 0.54


## VGG16

In [ ]:
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False)

In [ ]:
# checking the type
type(vgg16_model)

In [ ]:
# Create your own input format 
X_input = Input(shape=(256,256,3),name = 'image_input')

In [ ]:
output_vgg16_conv = vgg16_model(X_input)

In [ ]:
# Add the fully-connected layers 
X = Flatten(name='flatten')(output_vgg16_conv)
X = Dense(2048, activation='relu', name='fc1')(X)
X = Dense(2048, activation='relu', name='fc2')(X)
X = Dense(2, activation='softmax', name='predictions')(X)

In [ ]:
my_model = Model(inputs=X_input, outputs=X,name='myModel')

In [ ]:
my_model.summary()

In [ ]:
my_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
my_model.fit(x = X_train, y = Y_train, epochs = 50, batch_size = 32, validation_data=(X_test,Y_test))

In [ ]:
preds = my_model.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

## InceptionV3

In [ ]:
inceptionV3_model = keras.applications.inception_v3.InceptionV3(weights='imagenet',include_top=False)

In [ ]:
type(inceptionV3_model)

In [ ]:
# Freeze all the other layers, let last few layers to be trainable
for layer in inceptionV3_model.layers[:290]:
    layer.trainable = False

In [ ]:
# Create your own input format
X_input = Input(shape=(256,256,3),name = 'image_input')

In [ ]:
output_inceptionV3_conv = inceptionV3_model(X_input)

In [ ]:
# keras recommended application
X = GlobalAveragePooling2D()(output_inceptionV3_conv)
X = Dense(1024, activation='relu')(X)
X = Dense(4, activation='softmax', name='predictions')(X)

In [ ]:
my_model = Model(inputs=X_input, outputs=X,name='myModel')

In [ ]:
my_model.summary()

In [ ]:
my_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
my_model.fit(x = X_train, y = Y_train, epochs = 50, batch_size = 32, validation_data=(X_test,Y_test))

In [ ]:
preds = my_model.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

## Resnet50

In [ ]:
# To resolve the issue of BN in keras layers
K.set_learning_phase(1)

In [ ]:
resnet50_model = keras.applications.resnet50.ResNet50(weights='imagenet',include_top=False)

In [ ]:
type(resnet50_model)

In [ ]:
# Freeze all the other layers
for layer in resnet50_model.layers:
    layer.trainable = False

In [ ]:
# Create your own input format 
X_input = Input(shape=(256,256,3),name = 'image_input')

In [ ]:
output_resnet50_conv = resnet50_model(X_input)

In [ ]:
X = GlobalAveragePooling2D()(output_resnet50_conv)
# X = Dense(4, activation='softmax', name='predictions')(X)
X = Flatten(name='flatten')(output_resnet50_conv)
X = Dense(2048, activation='relu', name='fc1')(X)
X = Dropout(0.7)(X)
X = Dense(1024, activation='relu', name='fc2')(X)
X = Dropout(0.7)(X)
X = Dense(2, activation='softmax', name='predictions')(X)

In [ ]:
my_model = Model(inputs=X_input, outputs=X,name='myModel')

In [ ]:
my_model.summary()

In [ ]:
my_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
my_model.fit(x = X_train, y = Y_train, epochs = 50, batch_size = 32, validation_data=(X_test,Y_test))

In [ ]:
preds = my_model.evaluate(x = X_test, y = Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))